# ECVP 2024 - stimupy tutorial

Useful links:
- stimupy readthedocs: https://stimupy.readthedocs.io/en/latest/index.html
- stimupy paper: https://doi.org/10.21105/joss.05321
- stimupy github: https://github.com/computational-psychology/stimupy

# Organization of stimupy

More information: https://stimupy.readthedocs.io/en/latest/topic_guides/organization.html

## Overview of modules and their content

stimupy's building blocks are the following five modules:

In [ ]:
from stimupy import utils as utils               # Utility functions (plotting, exporting, etc)
from stimupy import components as components     # Stimulus components (shapes, texts, gratings, etc)
from stimupy import stimuli as stimuli           # Stimuli (illusions, Gabors, plaids, etc)
from stimupy import noises as noises             # Noise textures
from stimupy import papers as papers             # Stimulus sets of papers (full parametrizations + data)

### Utility functions
stimupy contains a large number of utility functions which can be used to manipulate (e.g. pad, adapt contrasts, filter, mask), plot, and/or export the stimulus arrays that it can generate.
All of these functions are listed and documented extensively here: https://stimupy.readthedocs.io/en/latest/reference/_api/stimupy.utils.html

In the following tutorial, we will mostly use the plotting utilities of stimupy.

In [ ]:
# Get list with all submodules using inspect library
import inspect
print("Submodules:", [x[0] for x in inspect.getmembers(utils, inspect.ismodule)])

In [ ]:
# List all attributes / functions of utils
#dir(utils)

### Components
There is no objective way to differentiate stimupy's components from its stimuli.
In principle, every component could be considered a stimulus.
The distinctions we make in stimupy are: firstly, that the components are “atomic” in a sense and hence underlie multiple different stimuli; secondly, most stimuli contain target(s) – a region of special scientific interest –, and come with a target_mask that indicates these targets.

A full overview of components and their documentation can be found here: https://stimupy.readthedocs.io/en/latest/reference/_api/stimupy.components.html

In the following, we will provide a quick overview of stimupy's components as implemented in `stimupy.components.plot_overview()`

In [ ]:
help(components)

In [ ]:
components.plot_overview()

### Stimuli

A full overview of stimuli and their documentation can be found here: https://stimupy.readthedocs.io/en/latest/reference/_api/stimupy.stimuli.html

In the following, we will provide a quick overview of stimupy's stimuli as implemented in `stimupy.stimuli.plot_overview()`

In [ ]:
help(stimuli)

In [ ]:
import warnings
warnings.filterwarnings('ignore') # Turn off warnings for now

stimuli.plot_overview()

## Noises

stimupy also provides the most commonly used noise textures. A full overiew with documentation can be found here: https://stimupy.readthedocs.io/en/latest/reference/_api/stimupy.noises.html

In [ ]:
help(noises)

In [ ]:
noises.plot_overview()

## Papers

Finally, stimupy comes with a growing list of exact re-implementations of stimulus sets from existing publications. These re-implementations allow stimupy users to import full stimulus sets including full parameterizations but also including e.g. experimental data within a single line of code.
A full overview of existing paper-reimplementations can be found here: https://stimupy.readthedocs.io/en/latest/reference/_api/stimupy.papers.html

In the following, we show the ModelFest stimuli as one example of stimupy's versatility.

In [ ]:
print(papers.__all__)

In [ ]:
from stimupy.papers import modelfest      # papers need to be imported explicitly
modelfest_stims = modelfest.gen_all()     # generate modelfest stimuli

In [ ]:
utils.plot_stimuli(modelfest_stims, units="deg")  # Plot all modelfest stimuli with utils

# How does stimupy work?

In [ ]:
warnings.filterwarnings('always') # Turn on warnings again for useful information

## Generating a stimulus

### Interaction: what should be the input arguments?

### Stimulus function documentation

In [ ]:
help(stimuli.whites.white)

### Create stimulus array

In [ ]:
# Create stimulus
white = stimuli.whites.white(
    "visual_size": 5.,
    "ppd": 32,
    "n_bars": 10,
    "target_indices": (3, -2),
    "target_heights": 1.,
)

# Plot stimulus
utils.plot_stim(white, units="deg")

## The output dictionary

There are two important things to highlight regarding stimupy's outputs:
1. stimupy-functions return a `dict`. These dictionaries contain the stimulus array (`"img"`), information about the stimulus as well as a number of masks that we will explore later. One of the advantages of using dictionaries is that we can add as much additional information as we need.
For the paper-stimuli that we have seen previously, this can also be e.g. experimental data.
2. The stimulus is provided as an array (`"img"`). This allows the user to use any standard array-manipulation tooling to further process / maniupulate the stimulus.


In [ ]:
print("Type:", type(white))
print()
print("Keys:", white.keys())

In [ ]:
# Add function which strips arguments which cannot be fed into function here.

### Filling in the blanks

One important feature to highlight: stimupy filled in all of the blanks, i.e. all of the parameters that we did not specify but which can be inferred.

In `stimupy.stimuli.whites.white`, we passed a `visual_size`, a `ppd` and `n_bars`, and stimupy inferred the `shape`, `frequency` and `bar_width` of the stimulus.

In [ ]:
for k in white.keys():
    if k not in ["img", "grating_mask", "bar_mask", "target_mask"]: print(k + ":", white[k])

### Obsession with masks
The output `dict` also contains several `mask`-keys.

Just like the `"img"`, a stimulus `"mask"` is a numpy.ndarray. 
Each entry of the `"mask"` corresponds to a pixel in the stimulus array (i.e., it has the same shape).

Importantly, the `"mask"` contains only integer-values which correspond to a geometric region of interest.
In our case, we have a `"target_mask"` and a `"bar_mask"`.
We can see that the masks allow us to specifically interact with the corresponding regions of interest.
This is particularly useful for computational modeling or for stimulus composition.

Find more information on stimulus composition with stimupy here: https://stimupy.readthedocs.io/en/latest/getting_started/composition.html

In [ ]:
utils.plot_stim(white, units="deg", mask="target_mask") # Plot stimulus with target mask
utils.plot_stim(white, units="deg", mask="bar_mask")    # Plot stimulus with bar mask

### Demo: adding a noise texture

To demonstrate the utility of masks, we will use the `"bar_mask"` to add a noise texture on all black bars of White's stimulus.

In [ ]:
# Create pink noise
pinkNoise = noises.naturals.pink(
    visual_size=white["visual_size"],
    ppd=white["ppd"],
    intensity_range=(-.5, .5)
)

# Plot stimuli
utils.plot_stim(pinkNoise, vmin=-.5, vmax=.5)

In [ ]:
import numpy as np

# Use bar_mask to add noise to black bars only
noisyImg = np.where(
    ((white["bar_mask"]%2==0) * (white["target_mask"]==0)),
    white["img"]+pinkNoise["img"],
    white["img"]
)

# Create stimulus dictionary with all information
noisyWhite = {**white, **pinkNoise}
noisyWhite["img"] = noisyImg
print(noisyWhite.keys())

In [ ]:
utils.plot_stimuli({"white": white, "noisyWhite": noisyWhite}, vmin=-.5, vmax=1.5)

# Parameterizing stimuli in stimupy

## Interactively

## Manually (above example)

In [ ]:
# Define parameters manually
whiteParams = {
    "visual_size": 5.,
    "ppd": 32,
    "n_bars": 10,
    "target_indices": (3, -2),
    "target_heights": 1.,
}

white = stimuli.whites.white(**whiteParams)   # Create stimulus
utils.plot_stim(white, units="deg")           # Plot stimulus

## Interactively

In [ ]:
import ipywidgets as iw
from stimupy.utils import plot_stim
from stimupy.stimuli.whites import white

# Define widgets
w_height = iw.IntSlider(value=5, min=1, max=20, description="height [deg]")
w_width = iw.IntSlider(value=5, min=1, max=20, description="width [deg]")
w_ppd = iw.IntSlider(value=20, min=1, max=40, description="ppd")

w_freq = iw.FloatSlider(value=1, min=0, max=2, description="frequency [cpd]")
w_rot = iw.FloatSlider(value=0, min=0, max=360, description="rotation [deg]")

w_int1 = iw.FloatSlider(value=1, min=0, max=1, description="int1")
w_int2 = iw.FloatSlider(value=0, min=0, max=1, description="int2")

w_tidx = iw.IntSlider(value=3, min=0, max=20, description="target idx")
w_tint = iw.FloatSlider(value=0.5, min=0, max=1, description="target int")
w_theights = iw.FloatSlider(value=1, min=0, max=5, description="target heights [deg]")

w_tidx2 = iw.IntSlider(value=-2, min=-20, max=0, description="target2 idx")
w_tint2 = iw.FloatSlider(value=0.5, min=0, max=1, description="target2 int")
w_theights2 = iw.FloatSlider(value=1, min=0, max=5, description="target2 heights [deg]")

w_ori = iw.Dropdown(value="corner", options=['mean', 'corner', 'center'], description="origin")
w_period = iw.Dropdown(value="ignore", options=['ignore', 'even', 'odd', 'either'], description="period")
w_mask = iw.ToggleButton(value=False, disabled=False, description="add mask")

# Layout
b_im_size = iw.HBox([w_height, w_width, w_ppd])
b_geometry = iw.HBox([w_freq, w_rot])
b_intensities = iw.HBox([w_int1, w_int2])
b_target = iw.HBox([w_tidx, w_tint, w_theights])
b_target2 = iw.HBox([w_tidx2, w_tint2, w_theights2])
b_add = iw.HBox([w_ori, w_period, w_mask])
ui = iw.VBox([b_im_size, b_geometry, b_intensities, b_target, b_target2, b_add])

# Function for showing stim
def show_white(
    height=None,
    width=None,
    ppd=None,
    rotation=None,
    frequency=None,
    sigma=None,
    int1=None,
    int2=None,
    origin=None,
    period=None,
    add_mask=False,
    target_idx=None,
    intensity_target=None,
    target_idx2=None,
    intensity_target2=None,
    target_heights=None,
    target_heights2=None,
):
    try:
        stim = white(
            visual_size=(height, width),
            ppd=ppd,
            rotation=rotation,
            frequency=frequency,
            intensity_bars=(int1, int2),
            origin=origin,
            period=period,
            target_indices=(target_idx, target_idx2),
            intensity_target=(intensity_target, intensity_target2),
            target_heights=(target_heights, target_heights2),
        )
        plot_stim(stim, mask=add_mask)
    except Exception as e:
        raise ValueError(f"Invalid parameter combination: {e}") from None

# Set interactivity
out = iw.interactive_output(
    show_white,
    {
        "height": w_height,
        "width": w_width,
        "ppd": w_ppd,
        "rotation": w_rot,
        "frequency": w_freq,
        "int1": w_int1,
        "int2": w_int2,
        "origin": w_ori,
        "period": w_period,
        "add_mask": w_mask,
        "target_idx": w_tidx,
        "intensity_target": w_tint,
        "target_heights": w_theights,
        "target_idx2": w_tidx2,
        "intensity_target2": w_tint2,
        "target_heights2": w_theights2,
    },
)

# Show
display(ui, out)

## Stimulus parameter spaces (stim-spaces)

Stimspace example with white stimulus

In [ ]:
# Provide code with a parameter space

# Playground

We introduced you to the principles of stimupy.
Now, it's your turn.
Feel free to explore the many stimuli you can use with stimupy and personalize it.

Use the above introduced functions to guide your stimulus creation or ask us:
- `help(function)` to see documentation of function
- documentation of each function: https://stimupy.readthedocs.io/en/latest/reference/_api/stimupy.stimuli.html
- demos for each function: https://stimupy.readthedocs.io/en/latest/reference/demos.html

In [ ]:
# Print attributes
print("stimupy.stimuli attributes:\n", stimuli.__all__)
print()

# Print attributes of two example submodules
print("stimupy.stimuli.whites attributes:\n", stimuli.whites.__all__)
print()
print("stimupy.stimuli.sbcs attributes:\n", stimuli.sbcs.__all__)

In [ ]:
help(stimuli.sbcs.with_dots_two_sided)